# Script for running Machine-Learning models for Detecting Hot Spots
## Part 2 Inference

#### Requirements
* matplotlib
* numpy
* pandas
* pycaret
* rasterio

In [1]:
import glob
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd

import pycaret
import pycaret.classification as cl

import rasterio

#### Settings 

In [4]:
MODEL = '../models/2020-10-30_catBoost_tuned_model'
EXPERIMENT = '../models/2020-10-30_catBoost_tuned'
#MODEL = '../models/2020-11-03_ET_calibrated_7class_model'

In [ ]:
DATA = '../data_inference/Inference_Baldwin.tif'
DATA_OUTPUT = '../data_inference/Inference_Baldwin_classified.tif'

In [ ]:
DATA = '../data_inference/Yana/Inference_Yana-0000006400-0000019200-016.tif'
DATA_OUTPUT = '../data_inference/Yana/Inference_Yana-0000006400-0000019200-016_classified.tif'

#### Model Loading 

In [3]:
model = cl.load_model(MODEL)

Transformation Pipeline and Model Sucessfully Loaded


In [5]:
exp = cl.load_experiment(EXPERIMENT)

,Object
0,Classification Setup Config
1,X_training Set
2,y_training Set
3,X_test Set
4,y_test Set
5,Transformation Pipeline
6,Compare Models Score Grid
7,Tuned OneVsRestClassifier
8,Tuned OneVsRestClassifier Score Grid
9,Tuned OneVsRestClassifier


### Data Loading 
Load raster data and reorder

In [ ]:
with rasterio.open(DATA) as src:
    columns = src.descriptions
    data = src.read()

In [ ]:
data.shape

### Data Analysis - Prediction 

In [ ]:
data_reshaped = pd.DataFrame(data.reshape(27, -1).T, columns=columns)

In [ ]:
prediction = cl.predict_model(model, data_reshaped)['Label'].values.reshape(*data.shape[1:])

### Data Export 

In [ ]:
with rasterio.open(DATA) as src:
    with rasterio.open(DATA_OUTPUT, mode='w', 
                    driver=src.driver, 
                    width=src.width, 
                    height=src.height, 
                    count=1, 
                    dtype='uint8', 
                    transform=src.transform,
                    crs=src.crs) as dst:
        dst.write_band(1, np.array(prediction, dtype=np.uint8))

### Visualization

In [ ]:
plt.imshow(prediction)

In [ ]:
plt.imshow(prediction==15)

### Automation

In [6]:
def run_inference(model_path, infile_path, outfile_path):
    print("Processing dataset input:", infile_path)
    print("Processing dataset output:", outfile_path)
    
    model = cl.load_model(model_path)
    
    with rasterio.open(infile_path) as src:
        columns = src.descriptions
        data = src.read()
    data_reshaped = pd.DataFrame(data.reshape(27, -1).T, columns=columns)
    
    prediction = cl.predict_model(model, data_reshaped)['Label'].values.reshape(*data.shape[1:])
    
    with rasterio.open(infile_path) as src:
        with rasterio.open(outfile_path, 
                            mode='w', 
                            driver=src.driver, 
                            width=src.width, 
                            height=src.height, 
                            count=1, 
                            dtype='uint8', 
                            transform=src.transform,
                            crs=src.crs) as dst:
            dst.write_band(1, np.array(prediction, dtype=np.uint8))

In [8]:
flist = glob.glob(r'K:\127_HotSpotOptimizer\hot_spot_classifier\data_inference\*.tif')

In [9]:
flist[:2]

[]

In [10]:
for f in flist[:2]:
    outfile = os.path.join(os.path.dirname(f), os.path.basename(f)[:-4] + '_classified.tif')
    run_inference(MODEL, f, outfile)